<a href="https://colab.research.google.com/github/LuigiSigillo/Aviation-accidents-visualizations/blob/main/Data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data preprocessing

before preprocessing

In [1]:
import sys
import time
import os
import pandas as pd
from google.colab import drive

drive.mount('/content/drive')
#!unzip "/content/drive/My Drive/AviationData.csv.zip" -d "/content/drive/My Drive/"

df = pd.read_csv("/content/drive/My Drive/AviationCrashLocation.csv", encoding='ISO-8859-1')
df

Mounted at /content/drive


,Event.Id,Event.Date,Location,Aircraft.Damage,Make,Purpose.of.Flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.Phase.of.Flight,Crash.Location,Crash.Country
0,20200502X81549,2020-05-02,"PALMYRA, IL",Destroyed,Yakovlev,Personal,1.0,0.0,0.0,0.0,VMC,MANEUVERING,"39.408889,-89.990277",Illinois
1,20200413X13054,2020-04-11,"Eagle River, AK",Substantial,Cessna,Personal,0.0,2.0,2.0,0.0,VMC,MANEUVERING,"61.351943999999996,-149.548889",Alaska
2,20200326X82317,2020-03-25,"Waxahachie, TX",Substantial,Cessna,Personal,0.0,1.0,0.0,0.0,VMC,STANDING,"32.448055,-96.913889",Texas
3,20200324X34735,2020-03-23,"Swansboro, NC",Destroyed,Maule,Personal,2.0,0.0,0.0,0.0,IMC,UNKNOWN,"34.416945,-77.034166",Unknown
4,20200318X22054,2020-03-18,"Eagle Creek, OR",Destroyed,Piper,Personal,0.0,1.0,0.0,0.0,VMC,TAKEOFF,"45.352778,-122.340833",Oregon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,20010417X00765,2001-04-09,"Vero Beach, FL",Destroyed,Piper,Personal,2.0,0.0,0.0,0.0,VMC,TAKEOFF,"27.649721999999997,-80.439166",Florida
2624,20010206X00396,2001-01-24,"Nashville, TN",Destroyed,Beech,Business,4.0,0.0,0.0,0.0,VMC,TAKEOFF,"36.13,-86.653889",Tennessee
2625,20010119X00328,2001-01-14,"Troy, AL",Destroyed,Learjet,Executive/Corporate,0.0,2.0,0.0,0.0,VMC,LANDING,"31.866667,-86.016667",Alabama
2626,20001212X20873,2000-04-05,"MARIANNA, FL",Destroyed,LEARJET,Instructional,3.0,0.0,0.0,0.0,VMC,APPROACH,"1.483333,-75.533333",CaquetÃ¡


# After preprocessing

In [ ]:
import numpy as np
df = pd.read_csv("/content/drive/My Drive/AviationData.csv", encoding='ISO-8859-1')

#View the dataset
print("1, intero", df.shape, "TOT = ", df.shape[0]*df.shape[1])



df = df.drop(
    columns=["Accident.Number","Airport.Code","Airport.Name","Registration.Number","Engine.Type","FAR.Description","Schedule","Air.Carrier","Report.Status","Publication.Date"]
)
print("2, senza colonne inutili", df.shape, "TOT = ", df.shape[0]*df.shape[1])



print("__________________________________________")
df = df.drop(df[df["Investigation.Type"] == "Incident"].index)
df=df.drop(columns=["Investigation.Type"])

print("3, senza incidents", df.shape, "TOT = ", df.shape[0]*df.shape[1])



print("__________________________________________")
df = df[df["Latitude"].notnull()]
df = df[df["Longitude"].notnull()]

print("4, solo coordinate", df.shape, "TOT = ", df.shape[0]*df.shape[1])


print("__________________________________________")
## Da mettere solo quelli con fatal (#num morti) o non fatal
patternDel = ".*Fatal.*"
filter = df["Injury.Severity"].str.contains(patternDel)
df = df[filter]

print("5, solo severity", df.shape, "TOT = ", df.shape[0]*df.shape[1])


print("__________________________________________")
df = df[df["Aircraft.Damage"].notnull()]
print("6, solo damage", df.shape, "TOT = ", df.shape[0]*df.shape[1])
df=df.drop(columns=["Aircraft.Category"])

'''
print("__________________________________________")
df = df[df["Aircraft.Category"]=="Airplane"]

print("7, solo airplane", df.shape, "TOT = ", df.shape[0]*df.shape[1])

print("__________________________________________")
df = df[df["Amateur.Built"]=="No"]

print("8, solo non amatoriali", df.shape, "TOT = ", df.shape[0]*df.shape[1])
'''
df=df.drop(columns=["Amateur.Built"])
print("__________________________________________")
df = df[df["Purpose.of.Flight"].notnull()]
df = df[df["Purpose.of.Flight"]!="Unknown"]
print("9, solo con purpose", df.shape, "TOT = ", df.shape[0]*df.shape[1])

print("__________________________________________")
df = df.dropna(subset=['Total.Fatal.Injuries','Total.Serious.Injuries'], how='all')
print("10, tutti NAN a casa", df.shape, "TOT = ", df.shape[0]*df.shape[1])


print("__________________________________________")
df = df[df["Weather.Condition"].notnull()]
print("11, solo con wether", df.shape, "TOT = ", df.shape[0]*df.shape[1])

print("__________________________________________")
df = df[df["Broad.Phase.of.Flight"].notnull()]
print("12, solo board phase", df.shape, "TOT = ", df.shape[0]*df.shape[1])


print("__________________________________________")
df = df[df["Number.of.Engines"].notnull()]
df=df.drop(columns=["Number.of.Engines"])

print("13, solo motore", df.shape, "TOT = ", df.shape[0]*df.shape[1])


print("__________________________________________")
df = df[df["Model"].notnull()]
df = df[df["Make"].notnull()]
print("14, not nan make e model", df.shape, "TOT = ", df.shape[0]*(df.shape[1]-3))



print("__________________________________________")
df =  df.drop(df[df["Country"] != "United States"].index)
df=df.drop(columns=["Country"])
print("15, only USA", df.shape, "TOT = ", df.shape[0]*(df.shape[1]-3))



df=df.drop(columns=["Injury.Severity"])
df=df.drop(columns=["Model"])
df["Crash.Location"] = df["Latitude"].astype(str) + "," + df["Longitude"].astype(str) 
df=df.drop(columns=["Latitude"])
df=df.drop(columns=["Longitude"])
df['Total.Fatal.Injuries'] = df['Total.Fatal.Injuries'].fillna(0)
df = df.fillna(0)


"""
print("__________________________________________")
df = df.drop(df[df["Injury.Severity"] == "Non-Fatal"].index)
df = df.drop(df[df["Injury.Severity"] == "Unavailable"].index)
print("16, solo fatali", df.shape, "TOT = ", df.shape[0]*df.shape[1]-3)
print(df.head())
"""
"""
print("__________________________________________")
df = df.drop(df[df["Event.Date"] < "2015-01-01"].index)
df = df.drop(df[df["Injury.Severity"] == "Unavailable"].index)
print("4, dal 2000 in poi", df.shape, "TOT = ", df.shape[0]*(df.shape[1]-3))
print(df.head())
"""


print("TOT = ", df.shape[0]*(df.shape[1]))
#df.to_csv(r'/content/drive/My Drive/Aviation cleaned.csv', index = False)
df.describe()
df

# Coordinates to Country

In [ ]:
import requests
from bs4 import BeautifulSoup
apiurl = "http://api.geonames.org/countrySubdivision?"
#lat=42.561944&lng=-71.77166700000001&username=LENZA
lat = "&lat="
lng = "&lng="
user = "&username=LENZA"




for index, row in df.iterrows():
    latlng = row['Crash.Location']

    print(str(index) + " | " + row['Crash.Location'])
    # index -> indice di riga
    # row['Crash.Location'] -> latitudine,longitudine
    latitude = latlng.split(",")[0]
    longitude = latlng.split(",")[1]
    #print(latitude + " | " + longitude)
    r = requests.get(apiurl + lat + latitude + lng + longitude + user)
    print(r)
    soup = BeautifulSoup(r.content, 'html.parser')
    #print(soup.prettify())
    try:
        print(soup.adminname1.string)
        df.loc[index, 'Crash.Country'] = soup.adminname1.string
    except:
        df.loc[index, 'Crash.Country'] = "Unknown"
    #df[index]['Crash.Country'] = soup.adminname1.string
    

df


In [ ]:
df

In [ ]:
print("TOT = ", df.shape[0]*(df.shape[1]))
df.to_csv(r'/content/drive/My Drive/AviationCrashLocation_new.csv', index = False)
df.describe()
df

# Aggregation

,Event.Id,Event.Date,Location,Aircraft.Damage,Make,Purpose.of.Flight,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.Phase.of.Flight,Crash.Location,Crash.Country
0,20200502X81549,2020-05-02,"PALMYRA, IL",Destroyed,Yakovlev,Personal,1.0,0.0,0.0,0.0,VMC,MANEUVERING,"39.408889,-89.990277",Illinois
1,20200413X13054,2020-04-11,"Eagle River, AK",Substantial,Cessna,Personal,0.0,2.0,2.0,0.0,VMC,MANEUVERING,"61.351943999999996,-149.548889",Alaska
2,20200326X82317,2020-03-25,"Waxahachie, TX",Substantial,Cessna,Personal,0.0,1.0,0.0,0.0,VMC,STANDING,"32.448055,-96.913889",Texas
3,20200324X34735,2020-03-23,"Swansboro, NC",Destroyed,Maule,Personal,2.0,0.0,0.0,0.0,IMC,UNKNOWN,"34.416945,-77.034166",Unknown
4,20200318X22054,2020-03-18,"Eagle Creek, OR",Destroyed,Piper,Personal,0.0,1.0,0.0,0.0,VMC,TAKEOFF,"45.352778,-122.340833",Oregon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2623,20010417X00765,2001-04-09,"Vero Beach, FL",Destroyed,Piper,Personal,2.0,0.0,0.0,0.0,VMC,TAKEOFF,"27.649721999999997,-80.439166",Florida
2624,20010206X00396,2001-01-24,"Nashville, TN",Destroyed,Beech,Business,4.0,0.0,0.0,0.0,VMC,TAKEOFF,"36.13,-86.653889",Tennessee
2625,20010119X00328,2001-01-14,"Troy, AL",Destroyed,Learjet,Executive/Corporate,0.0,2.0,0.0,0.0,VMC,LANDING,"31.866667,-86.016667",Alabama
2626,20001212X20873,2000-04-05,"MARIANNA, FL",Destroyed,LEARJET,Instructional,3.0,0.0,0.0,0.0,VMC,APPROACH,"1.483333,-75.533333",CaquetÃ¡


## Aggregate per State

In [44]:
'''
per ogni stato serve
Incidents -> entries per stato
Total.Fatal.Injuries
Total.Serious.Injuries	
Total.Minor.Injuries
Total.Uninjured
Weather.Condition
Aircraft.Damage
Broad.Phase.of.Flight
Make
Months
Distance
'''
df = pd.read_csv("/content/drive/My Drive/AviationCrashLocation.csv", encoding='ISO-8859-1')
groups = df.groupby(by='Crash.Country')
groups.groups
final_dict = {}
for group in groups.groups:
    final_dict[group] = {}
    final_dict[group]['Incidents'] = len(groups.groups[group])
    final_dict[group]['Total.Fatal.Injuries'] = 0
    final_dict[group]['Total.Serious.Injuries'] = 0
    final_dict[group]['Total.Minor.Injuries'] = 0
    final_dict[group]['Total.Uninjured'] = 0
    final_dict[group]['Weather.Condition'] = {}
    final_dict[group]['Aircraft.Damage'] = {}
    final_dict[group]['Broad.Phase.of.Flight'] = {}
    final_dict[group]['Make'] = {}
    final_dict[group]['Month'] = {}

    for index in groups.groups[group]:
        final_dict[group]['Total.Fatal.Injuries'] += df['Total.Fatal.Injuries'][index]
        final_dict[group]['Total.Serious.Injuries'] += df['Total.Serious.Injuries'][index]
        final_dict[group]['Total.Minor.Injuries'] += df['Total.Minor.Injuries'][index]
        final_dict[group]['Total.Uninjured'] += df['Total.Uninjured'][index]
        try:
            final_dict[group]['Weather.Condition'][df['Weather.Condition'][index]] += 1
        except:
            final_dict[group]['Weather.Condition'][df['Weather.Condition'][index]] = 1

        try:
            final_dict[group]['Aircraft.Damage'][df['Aircraft.Damage'][index]] += 1
        except:
            final_dict[group]['Aircraft.Damage'][df['Aircraft.Damage'][index]] = 1

        try:
            final_dict[group]['Broad.Phase.of.Flight'][df['Broad.Phase.of.Flight'][index]] += 1
        except:
            final_dict[group]['Broad.Phase.of.Flight'][df['Broad.Phase.of.Flight'][index]] = 1
        
        try:
            final_dict[group]['Make'][df['Make'][index].lower()] += 1
        except:
            final_dict[group]['Make'][df['Make'][index].lower()] = 1

        try:
            final_dict[group]['Month'][df['Event.Date'][index].split('-')[1]] += 1
        except:
            final_dict[group]['Month'][df['Event.Date'][index].split('-')[1]] = 1


print(str(final_dict))

{'Adjuntas': {'Incidents': 1, 'Total.Fatal.Injuries': 2.0, 'Total.Serious.Injuries': 0.0, 'Total.Minor.Injuries': 0.0, 'Total.Uninjured': 0.0, 'Weather.Condition': {'IMC': 1}, 'Aircraft.Damage': {'Substantial': 1}, 'Broad.Phase.of.Flight': {'DESCENT': 1}, 'Make': {'partenavia': 1}, 'Month': {'01': 1}}, 'Alabama': {'Incidents': 43, 'Total.Fatal.Injuries': 45.0, 'Total.Serious.Injuries': 22.0, 'Total.Minor.Injuries': 9.0, 'Total.Uninjured': 6.0, 'Weather.Condition': {'VMC': 31, 'IMC': 12}, 'Aircraft.Damage': {'Destroyed': 13, 'Substantial': 29, 'Minor': 1}, 'Broad.Phase.of.Flight': {'TAKEOFF': 12, 'CLIMB': 1, 'MANEUVERING': 6, 'APPROACH': 11, 'GO-AROUND': 2, 'DESCENT': 1, 'CRUISE': 6, 'LANDING': 3, 'UNKNOWN': 1}, 'Make': {'piper': 11, 'champion': 2, 'cessna': 10, 'grob aircraft ag': 1, 'cirrus design corp': 2, 'beech': 7, 'israel aircraft industries': 1, 'republic': 1, 'aviat aircraft inc': 1, 'aero vodochody': 1, 'aero adventure': 1, 'nanchang china': 1, 'colyaer sl': 1, 'raytheon aircr

In [48]:
import json
with open("/content/drive/My Drive/aggregated_state.json", "w") as write_file:
    json.dump(final_dict, write_file, indent=4)

## Aggregate per Make

In [53]:
'''
per ogni stato serve
Incidents -> entries per stato
Total.Fatal.Injuries
Total.Serious.Injuries	
Total.Minor.Injuries
Total.Uninjured
Weather.Condition
Aircraft.Damage
Broad.Phase.of.Flight
Make
Months
Distance
'''
#df.apply(lambda x: x['Make'].astype(str).str.lower())
df = pd.read_csv("/content/drive/My Drive/AviationCrashLocation.csv", encoding='ISO-8859-1')
groups = df.groupby(df['Make'].str.lower())
final_dict = {}
for group in groups.groups:
    final_dict[group] = {}
    final_dict[group]['Incidents'] = len(groups.groups[group])
    final_dict[group]['Total.Fatal.Injuries'] = 0
    final_dict[group]['Total.Serious.Injuries'] = 0
    final_dict[group]['Total.Minor.Injuries'] = 0
    final_dict[group]['Total.Uninjured'] = 0
    final_dict[group]['Weather.Condition'] = {}
    final_dict[group]['Aircraft.Damage'] = {}
    final_dict[group]['Broad.Phase.of.Flight'] = {}
    final_dict[group]['Crash.Country'] = {}
    final_dict[group]['Month'] = {}

    for index in groups.groups[group]:
        final_dict[group]['Total.Fatal.Injuries'] += df['Total.Fatal.Injuries'][index]
        final_dict[group]['Total.Serious.Injuries'] += df['Total.Serious.Injuries'][index]
        final_dict[group]['Total.Minor.Injuries'] += df['Total.Minor.Injuries'][index]
        final_dict[group]['Total.Uninjured'] += df['Total.Uninjured'][index]
        try:
            final_dict[group]['Weather.Condition'][df['Weather.Condition'][index]] += 1
        except:
            final_dict[group]['Weather.Condition'][df['Weather.Condition'][index]] = 1

        try:
            final_dict[group]['Aircraft.Damage'][df['Aircraft.Damage'][index]] += 1
        except:
            final_dict[group]['Aircraft.Damage'][df['Aircraft.Damage'][index]] = 1

        try:
            final_dict[group]['Broad.Phase.of.Flight'][df['Broad.Phase.of.Flight'][index]] += 1
        except:
            final_dict[group]['Broad.Phase.of.Flight'][df['Broad.Phase.of.Flight'][index]] = 1
        
        try:
            final_dict[group]['Crash.Country'][df['Crash.Country'][index]] += 1
        except:
            final_dict[group]['Crash.Country'][df['Crash.Country'][index]] = 1

        try:
            final_dict[group]['Month'][df['Event.Date'][index].split('-')[1]] += 1
        except:
            final_dict[group]['Month'][df['Event.Date'][index].split('-')[1]] = 1


print(str(final_dict))

{'aero adventure': {'Incidents': 1, 'Total.Fatal.Injuries': 1.0, 'Total.Serious.Injuries': 0.0, 'Total.Minor.Injuries': 0.0, 'Total.Uninjured': 0.0, 'Weather.Condition': {'VMC': 1}, 'Aircraft.Damage': {'Substantial': 1}, 'Broad.Phase.of.Flight': {'TAKEOFF': 1}, 'Crash.Country': {'Alabama': 1}, 'Month': {'09': 1}}, 'aero at sp zoo': {'Incidents': 1, 'Total.Fatal.Injuries': 0.0, 'Total.Serious.Injuries': 1.0, 'Total.Minor.Injuries': 0.0, 'Total.Uninjured': 0.0, 'Weather.Condition': {'VMC': 1}, 'Aircraft.Damage': {'Substantial': 1}, 'Broad.Phase.of.Flight': {'GO-AROUND': 1}, 'Crash.Country': {'New Hampshire': 1}, 'Month': {'05': 1}}, 'aero commander': {'Incidents': 22, 'Total.Fatal.Injuries': 12.0, 'Total.Serious.Injuries': 16.0, 'Total.Minor.Injuries': 2.0, 'Total.Uninjured': 5.0, 'Weather.Condition': {'VMC': 21, 'IMC': 1}, 'Aircraft.Damage': {'Substantial': 18, 'Destroyed': 4}, 'Broad.Phase.of.Flight': {'APPROACH': 3, 'MANEUVERING': 6, 'LANDING': 2, 'STANDING': 1, 'DESCENT': 2, 'TAKEOFF

In [54]:
import json
with open("/content/drive/My Drive/aggregated_make.json", "w") as write_file:
    json.dump(final_dict, write_file, indent=4)

## Aggregate per month

In [ ]:
'''
per ogni stato serve
Incidents -> entries per stato
Total.Fatal.Injuries
Total.Serious.Injuries	
Total.Minor.Injuries
Total.Uninjured
Weather.Condition
Aircraft.Damage
Broad.Phase.of.Flight
Make
Months
Distance
'''
#df.apply(lambda x: x['Make'].astype(str).str.lower())
df = pd.read_csv("/content/drive/My Drive/AviationCrashLocation.csv", encoding='ISO-8859-1')
df = df.
groups = df.groupby(df['Event.Date'].str.split('-')[1])
final_dict = {}
for group in groups.groups:
    final_dict[group] = {}
    final_dict[group]['Incidents'] = len(groups.groups[group])
    final_dict[group]['Total.Fatal.Injuries'] = 0
    final_dict[group]['Total.Serious.Injuries'] = 0
    final_dict[group]['Total.Minor.Injuries'] = 0
    final_dict[group]['Total.Uninjured'] = 0
    final_dict[group]['Weather.Condition'] = {}
    final_dict[group]['Aircraft.Damage'] = {}
    final_dict[group]['Broad.Phase.of.Flight'] = {}
    final_dict[group]['Crash.Country'] = {}
    final_dict[group]['Make'] = {}

    for index in groups.groups[group]:
        final_dict[group]['Total.Fatal.Injuries'] += df['Total.Fatal.Injuries'][index]
        final_dict[group]['Total.Serious.Injuries'] += df['Total.Serious.Injuries'][index]
        final_dict[group]['Total.Minor.Injuries'] += df['Total.Minor.Injuries'][index]
        final_dict[group]['Total.Uninjured'] += df['Total.Uninjured'][index]
        try:
            final_dict[group]['Weather.Condition'][df['Weather.Condition'][index]] += 1
        except:
            final_dict[group]['Weather.Condition'][df['Weather.Condition'][index]] = 1

        try:
            final_dict[group]['Aircraft.Damage'][df['Aircraft.Damage'][index]] += 1
        except:
            final_dict[group]['Aircraft.Damage'][df['Aircraft.Damage'][index]] = 1

        try:
            final_dict[group]['Broad.Phase.of.Flight'][df['Broad.Phase.of.Flight'][index]] += 1
        except:
            final_dict[group]['Broad.Phase.of.Flight'][df['Broad.Phase.of.Flight'][index]] = 1
        
        try:
            final_dict[group]['Crash.Country'][df['Crash.Country'][index]] += 1
        except:
            final_dict[group]['Crash.Country'][df['Crash.Country'][index]] = 1

        try:
            final_dict[group]['Make'][df['Make'][index].lower()] += 1
        except:
            final_dict[group]['Make'][df['Make'][index].lower()] = 1


print(str(final_dict))